# Serial Dilution Demo

In [1]:
# Import libraries
from duckbot.Machine import Machine

In [2]:
# Connect to the machine, specifying your plate configuration
m = Machine()
m.set_plate(config='base_plate_config')

In [7]:
# We can take a look at our configured tools
m.plate.slots

{0: {'origin': [14.5, 91.2],
  'labware': 'opentrons_96_tiprack_300ul',
  'slot_index': 0,
  'calibration_positions': {'A1': [28.5, 77.0],
   'A12': [127.4, 76.1],
   'H12': [127.1, 13.5]},
  'well_count': 96,
  'row_count': 8,
  'column_count': 12,
  'labware_width': 98.90409496072445,
  'labware_height': 62.60071884571294,
  'x_spacing': 8.991281360065859,
  'y_spacing': 8.942959835101849,
  'theta': -0.006946072753633755,
  'wells': {'A1': [28.5, 77.0],
   'B1': [28.5, 68.05704016489815],
   'C1': [28.5, 59.1140803297963],
   'D1': [28.5, 50.17112049469445],
   'E1': [28.5, 41.228160659592604],
   'F1': [28.5, 32.28520082449076],
   'G1': [28.5, 23.342240989388905],
   'H1': [28.5, 14.399281154287053],
   'A2': [37.491281360065855, 77.0],
   'B2': [37.491281360065855, 68.05704016489815],
   'C2': [37.491281360065855, 59.1140803297963],
   'D2': [37.491281360065855, 50.17112049469445],
   'E2': [37.491281360065855, 41.228160659592604],
   'F2': [37.491281360065855, 32.28520082449076]

In [6]:
# Drop the bed plate down and load our labware
m.move_to(z=175)

# For our serial dilution demo, we'll need a pipette rack, a media reservoir, and a 96 well plate
tip_rack = m.plate.load_labware(0, "opentrons_96_tiprack_300ul")
reservoir = m.plate.load_labware(1, "scienfocus_6_wellplate")
well_plate = m.plate.load_labware(2, "scienfocus_96_wellplate")

Load opentrons_96_tiprack_300ul into slot 0. Press enter to continue.
Load scienfocus_6_wellplate into slot 1. Press enter to continue.
Load scienfocus_96_wellplate into slot 2. Press enter to continue.


In [8]:
# Grab the pipette
pipette = m.tool_change(1)

In [9]:
# Equip pipette with a pipette tip
pipette.pickup_tip(tip_rack)

In [10]:
# Make sure your sharps container is loaded in to dispose of tips
pipette.eject_tip()

In [11]:
# Next time we get a tip, we'll move to the next available one
pipette.pickup_tip(tip_rack)

In [ ]:
# Transfer 50 uL of water into each well
pipette.transfer(50, reservoir["wells"]["A1"], well_plate["wells"])

In [14]:
# Add 100 uL of solution to the first row
pipette.transfer(100, reservoir["wells"]["B1"], well_plate["columns"][1], mix_after = (3, 50))

In [ ]:
# Move 100 mL from the first well in the row to the next & mix, and so on
for row in well_plate["rows"]:
    for column in range(1, well_plate['column_count']):
        pipette.transfer(100, well_plate["wells"][f"{row}{column}"], well_plate["wells"][f"{row}{column+1}"], mix_after = (3, 50))

In [ ]:
# Dispose of your pipette tip
pipette.eject_tip()

In [ ]:
# Park the pipette
m.park_tool()

In [ ]:
########

In [ ]:
# Move to water and clear pipette
m.move_to(z=80)
water = m.plate.get_well_position(1, "A1")
m.move_to(x=water[0], y=water[1])
pipette.aspirate_prime()
pipette.blowout()

In [13]:
#
pipette.aspirate_prime()
pipette.blowout(40)

In [ ]:
# Move to blue solution and clear pipette
m.move_to(z=80)
blue = m.plate.get_well_position(1, "B1")
m.move_to(x=blue[0], y=blue[1])
pipette.aspirate_prime()
pipette.blowout()

In [ ]:
## Air Gap
blue = m.plate.get_well_position(1, "B1")
m.move_to(x=blue[0], y=blue[1])

for v in range(4):
    m.move_to(z=47)
    pipette.aspirate(20)
    pipette.air_gap()
    
m.move_to(z=80)